In [1]:
import os
os.environ["OPENAI_API_KEY"] ="paste your api key"


In [2]:
!pip install -q langchain-openai langchain-core requests

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

# tool create

In [4]:
@tool
def multiply(a:int,b:int) -> int:
    """Given 2 numbers a and b this tool returns their product"""
    return a*b

In [5]:
print(multiply.invoke({'a':3,'b':4}))

12


In [6]:
multiply.name

'multiply'

In [7]:
multiply.description


'Given 2 numbers a and b this tool returns their product'

In [8]:
multiply.args


{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

# tool binding

In [9]:
llm = ChatOpenAI()

In [10]:
llm.invoke('hi').content

'Hello! How can I help you today?'

In [11]:
llm_with_tools=llm.bind_tools([multiply])

In [12]:
llm_with_tools.invoke('Hi how are you').content

"Hello! I'm here and ready to assist you. How can I help you today?"

In [13]:
query = HumanMessage('can you multiply 3 with 1000')

In [14]:
messages = [query]

In [15]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [16]:
 result=llm_with_tools.invoke(messages)

In [17]:
messages.append(result)

In [18]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kzB23sHuYIOHNvHPSNoXdUdW', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 63, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CJfa0Fs9ye8ZR3DBqcnds5K9k8kdO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1521a71b-2c2f-4504-a5fe-e7b427c10814-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_kzB23sHuYIOHNvHPSNoXdUdW', 'type': 'tool_call'}], usage_metadata={'input_toke

In [19]:
tool_result=multiply.invoke(result.tool_calls[0])

In [20]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='call_kzB23sHuYIOHNvHPSNoXdUdW')

In [21]:
messages.append(tool_result)

In [22]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kzB23sHuYIOHNvHPSNoXdUdW', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 63, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CJfa0Fs9ye8ZR3DBqcnds5K9k8kdO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1521a71b-2c2f-4504-a5fe-e7b427c10814-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_kzB23sHuYIOHNvHPSNoXdUdW', 'type': 'tool_call'}], usage_metadata={'input_toke

In [23]:
llm_with_tools.invoke(messages).content

'The product of multiplying 3 with 1000 is 3000.'

# currency conversion tools

## tool create

In [24]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [25]:
@tool
def get_conversion_factor(base_currency:str, target_currency:str) ->float:
    """ This function fetches the currency conversion factor between a given base currency and a target currency """
    url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'
    response = requests.get(url)
    return response.json()
@tool
def convert(base_currency_value: int, conversion_rate:Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate
    
  
     


   

 

In [26]:
   convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [27]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1758758401,
 'time_last_update_utc': 'Thu, 25 Sep 2025 00:00:01 +0000',
 'time_next_update_unix': 1758844801,
 'time_next_update_utc': 'Fri, 26 Sep 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.789}

In [28]:
convert.invoke({'base_currency_value':10, 'conversion_rate':88.789})

887.89

# tool binding

In [29]:
llm=ChatOpenAI()

In [30]:
llm_with_tools=llm.bind_tools([get_conversion_factor,convert])

In [31]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [32]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [33]:
ai_message=llm_with_tools.invoke(messages)

In [34]:
messages.append(ai_message)

In [39]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iNwFNF4vomZoVW87cYau4vqQ', 'function': {'arguments': '{"base_currency": "INR", "target_currency": "USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_YQjhjA0IXu1prsJG6rUZWwxD', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 123, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CJfa3OgU9dg5wIcqQuDjA1aoARCYd', 'service_tier': 'defa

In [35]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_iNwFNF4vomZoVW87cYau4vqQ',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_YQjhjA0IXu1prsJG6rUZWwxD',
  'type': 'tool_call'}]

In [36]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [37]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iNwFNF4vomZoVW87cYau4vqQ', 'function': {'arguments': '{"base_currency": "INR", "target_currency": "USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_YQjhjA0IXu1prsJG6rUZWwxD', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 123, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CJfa3OgU9dg5wIcqQuDjA1aoARCYd', 'service_tier': 'defa

In [38]:
llm_with_tools.invoke(messages).content

'The conversion factor between INR and USD is 0.01126. \n\nBased on this, converting 10 INR to USD gives 0.1126 USD.'